# Exemplo 5: Smart City - Otimização de Tráfego (Dask)

Este notebook demonstra o uso de **Dask** para processar grandes volumes de dados de sensores de tráfego.

**Cenário**: Analisar a densidade de veículos em cruzamentos para otimizar o tempo dos semáforos.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Dados de Tráfego
Simulando dados de 50 cruzamentos principais ao longo de 24h.

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('traffic_data', exist_ok=True)

def generate_traffic_data(day):
    intersections = [f'INT-{i:03d}' for i in range(50)]
    periods = 1000
    
    df = pd.DataFrame({
        'intersection_id': np.random.choice(intersections, periods),
        'timestamp': pd.date_range(start=f'2024-01-{day:02d}', periods=periods, freq='min'),
        'vehicle_count': np.random.poisson(20, periods), # Média de 20 carros/min
        'avg_speed': np.random.normal(40, 10, periods)
    })
    
    # Simular Horário de Pico (mais carros, menor velocidade)
    # Simplificado: apenas aumentando contagem em alguns aleatórios
    df.loc[df['vehicle_count'] > 30, 'avg_speed'] = df['avg_speed'] * 0.5
    
    df.to_csv(f'traffic_data/day_{day}.csv', index=False)

print("Gerando dados de 7 dias...")
for i in range(1, 8):
    generate_traffic_data(i)

## 3. Processamento para Otimização

In [ ]:
ddf = dd.read_csv('traffic_data/day_*.csv')

# Agrupar por Interseção e calcular métricas
stats = ddf.groupby('intersection_id').agg({
    'vehicle_count': 'mean',
    'avg_speed': 'mean'
})

# Lógica de Otimização: Se contagem > 25, aumentar tempo de verde.
def optimize_green_light(row):
    base_time = 30 # segundos
    if row['vehicle_count'] > 25:
        return base_time + 15
    elif row['vehicle_count'] > 20:
        return base_time + 5
    else:
        return base_time

stats_computed = stats.compute()
stats_computed['green_light_sec'] = stats_computed.apply(optimize_green_light, axis=1)

print("Sugestão de Tempo de Semáforo (Top 10):")
print(stats_computed.sort_values('vehicle_count', ascending=False).head(10))

In [ ]:
client.close()